* Generating Tables for Spec Sources in the Northern Hemisphere after querying the chisqs of the rerun posteriors

In [1]:
import sys
import os
import h5py
import joblib
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

from astropy.io import fits
from functools import reduce
from astropy.table import Table

sys.path.append('../../methods_code_Nresol/')
sys.path.append('../../utils/')
import astro_cuts
import spec_utils

In [2]:
specdir = '/n/holylfs05/LABS/finkbeiner_lab/Lab/nmudur/bayestar_edr3/specmatched/'

In [3]:
dat = Table.read('lsd_scripts/spectrain_2_11_r50.fits', format='fits')

In [4]:
dat

_id,ra,dec,mag [8],mag_err [8],percentiles_E [3],percentiles_dm [3],gaia_id,l,b,plx,plx_err,chisq,gaia_edr3.source_id,gaia_edr3.l,gaia_edr3.b,gaia_edr3.pm,gaia_edr3.pmra,gaia_edr3.pmra_error,gaia_edr3.pmdec,gaia_edr3.pmdec_error,gaia_edr3.parallax,gaia_edr3.parallax_error,gaia_edr3.parallax_over_error,gaia_edr3.astrometric_n_good_obs_al,gaia_edr3.astrometric_chi2_al,gaia_edr3.astrometric_excess_noise,gaia_edr3.visibility_periods_used,gaia_edr3.phot_g_mean_mag,gaia_edr3.bp_rp,gaia_edr3.ruwe,gaia_dr2_source.l,gaia_dr2_source.b,gaia_dr2_source.ra,gaia_dr2_source.dec,gaia_dr2_source.ra_error,gaia_dr2_source.dec_error,gaia_dr2_source.solution_id,gaia_dr2_source.parallax,gaia_dr2_source.parallax_error,gaia_dr2_source.visibility_periods_used,gaia_dr2_source.astrometric_chi2_al,gaia_dr2_source.astrometric_n_good_obs_al,gaia_dr2_source.astrometric_excess_noise,gaia_dr2_source.phot_g_mean_mag,allwise.rchi2,allwise.w1rchi2,allwise.w1rchi2_pm,allwise.w1mpro,allwise.w1sigmpro,allwise.w2rchi2,allwise.w2mpro,allwise.w2sigmpro,allwise.w3rchi2,allwise.w3mpro,allwise.w3sigmpro,allwise.w4mpro,allwise.w4sigmpro,sdss_dr14_starsweep.psfflux [5],sdss_dr14_starsweep.psfflux_ivar [5],sdss_dr17_specobj.lsd_id,sdss_dr17_specobj.PLUG_RA,sdss_dr17_specobj.PLUG_DEC,sdss_dr17_specobj.CLASS,sdss_dr17_specobj.SUBCLASS,sdss_dr17_specobj.Z,sdss_dr17_specobj.Z_ERR,sdss_dr17_specobj.OBJID [5],sdss_dr17_specobj.ZWARNING,sdss_dr17_specobj.CHI68P,sdss_dr17_specobj.SN_MEDIAN_ALL,sdss_dr17_specobj.SN_MEDIAN [5],sdss_dr17_specobj.RCHI2
int64,float64,float64,float32,float32,float32,float32,int64,float64,float64,float32,float32,float32,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int16,float32,int32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float64,float64,bytes6,bytes21,float32,float32,int32,int32,float32,float32,float32,float32
3812121162708680749,166.8898991744587,14.031382641433913,19.151848 .. nan,0.022637922 .. nan,0.047559123 .. 0.12637307,14.44808 .. 15.170813,3968337099019056256,236.11459765232632,62.47441937595093,-0.97951996,0.3695996,4.7903857,3968337099019056256,236.11442440724588,62.47445126838801,0.5857974,-0.5589698115191557,0.32190564,0.17524666563387406,0.32441428,-0.8360367670470458,0.26492915,-3.1556993,204,213.54364,0.0,13,18.973822,0.59901047,1.0110707,236.11442449883526,62.47445126878341,166.88988627989121,14.031463044844516,0.2978273291179414,0.3121919936621859,1635721458409799680,-0.9795199675078325,0.3695996150515126,9,130.79942,140,0.0,18.944878,0.9695,1.079,1.079,15.949,0.059,0.8672,14.682,0.069,0.902,11.613,0.267,8.519,0.362,18.18577 .. 22.69877,4.1516204 .. 1.1326681,3812121162708683679,166.88989000000004,14.031464,QSO,BROADLINE,1.8418837,0.0002579228,3631 .. 186,0,1.040538,14.530279,6.1638155 .. 11.611566,2.3110104
3812121162708680756,166.91778214905796,14.001657195753234,19.265982 .. nan,0.034832116 .. nan,-1.1950498 .. -1.0782052,12.17514 .. 12.691472,0,236.20031892080314,62.48123232256002,nan,nan,1.542418,3968335823413832064,236.2001514714161,62.48125817739767,0.6720216,-0.5473344444627569,0.33602014,0.38992058303287164,0.2740193,-0.2941588006412824,0.26318276,-1.1176978,199,210.82132,0.47482842,13,19.010979,0.5106735,1.018414,236.20015173983955,62.481258205379504,166.91776547469814,14.001732004197486,0.3495083941586585,0.3424151681535966,1635721458409799680,-0.3309456341938689,0.42250557707508346,8,138.0282,131,0.40576751841814807,19.005375,0.9141,0.9578,0.9597,15.781,0.053,0.8546,14.442,0.059,0.9327,11.435,0.237,8.185,-9999.0,23.449995 .. 29.796429,3.0190814 .. 0.99915665,3812121162708681350,166.91779,14.001745,QSO,BROADLINE,1.5129158,0.0003891673,3631 .. 16,0,1.0136112,9.555128,6.216856 .. 4.981616,1.0492339
3812138754894725280,166.71577219196445,13.924149063868157,17.970095 .

In [5]:
fname = 'lsd_scripts/spectrain_2_11_r50.fits'
df_allspec = spec_utils.convert_to_dataframe_specmatched(fname)

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:41: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:45: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:47: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/spectra_matched/spec_utils.py:47: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]),

In [6]:
df_goodspec = spec_utils.return_goodspec(df_allspec)

Number of objects before / after 596315 210713
c1 = 0.873, c2 = 0.995, c3 = 0.445
Fraction left = 0.353


In [7]:
len(df_goodspec)

210713

In [8]:
len(np.unique(df_goodspec['gaia_edr3.source_id']))

210381

In [9]:
#get tiledict
import healpy as hp
pix256n = hp.ang2pix(256, df_goodspec['l'].to_numpy(), df_goodspec['b'].to_numpy(), lonlat=True, nest=True)
pix256n = np.unique(pix256n)
inputdir = '/n/holylfs05/LABS/finkbeiner_lab/Lab/nmudur/bayestar_edr3/input_fullsky/north/postprocessed/'
dirfiles = np.array(os.listdir(inputdir))
dirfiles = dirfiles[np.array([dirn.endswith('.h5') for dirn in dirfiles])]

In [10]:
dictmapper = spec_utils.get_pix_to_filemapper(inputdir, dirfiles, pix256n)

In [11]:
file2keys = spec_utils.invert_dict(dictmapper)

In [12]:
df_goodspec['pix256n'] = hp.ang2pix(256, df_goodspec['l'].to_numpy(), df_goodspec['b'].to_numpy(), lonlat=True, nest=True)

<ipython-input-12-382d7348b6da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goodspec['pix256n'] = hp.ang2pix(256, df_goodspec['l'].to_numpy(), df_goodspec['b'].to_numpy(), lonlat=True, nest=True)


In [13]:
df_goodspec['pix256n']

2         186397
3         186391
7         186421
9         186398
10        186396
           ...  
596290     22402
596291     22402
596296     22401
596298     22403
596305     22400
Name: pix256n, Length: 210713, dtype: int64

In [14]:
chisqrerun = np.ones(len(df_goodspec))*np.nan

In [15]:
'NumFiles', len(list(file2keys.keys()))

('NumFiles', 485)

In [16]:
for key in list(file2keys.keys()):
    #key = 'north.00475.h5'
    #opening only relevant files
    print(key)
    inputf = h5py.File(inputdir+key, 'r')
    allcatsrcs = []
    for pixel in file2keys[key]:#for each specmatched pixel
        dat = inputf[f'photometry/pixel 256-{pixel}'] #allcatdata
        allcatsrcs.append(np.vstack([dat['gaia.source_id'], dat['l'], dat['b'], dat['chisq']]).T)
    tab = np.vstack(allcatsrcs)
    specmask = df_goodspec['pix256n'].isin(file2keys[key]).to_numpy() 
    #all specmatsrcs in the specmatched pixels in that file
    specsrcs = df_goodspec.iloc[specmask]
    speclb = SkyCoord(l=specsrcs['l'].to_numpy()*u.deg, b= specsrcs['b'].to_numpy()*u.deg, frame='galactic')
    catlb = SkyCoord(l=tab[:, 1]*u.deg, b= tab[:, 2]*u.deg, frame='galactic')
    ispecincat, d2d, d3d = match_coordinates_sky(speclb, catlb)
    print(np.equal(tab[ispecincat, 0], specsrcs['gaia_edr3.source_id'].to_numpy()).sum()/len(ispecincat))
    chisqrerun[specmask] = tab[ispecincat, -1]
    inputf.close()

north.00475.h5
1.0
north.00466.h5
1.0
north.00943.h5
1.0
north.01317.h5
1.0
north.00909.h5
1.0
north.00979.h5
0.995575221238938
north.00707.h5
1.0
north.01067.h5
0.9992248062015504
north.00698.h5
0.9981481481481481
north.00531.h5
1.0
north.01196.h5
1.0
north.00789.h5
0.9974489795918368
north.00546.h5
0.9971181556195965
north.00974.h5
0.9975
north.01044.h5
0.9983443708609272
north.00910.h5
1.0
north.01359.h5
1.0
north.01080.h5
1.0
north.00961.h5
1.0
north.00438.h5
0.998062015503876
north.00325.h5
0.9972826086956522
north.00297.h5
1.0
north.00426.h5
1.0
north.01229.h5
1.0
north.01079.h5
1.0
north.00436.h5
1.0
north.01353.h5
1.0
north.00524.h5
0.9987531172069826
north.00822.h5
1.0
north.01245.h5
1.0
north.00294.h5
1.0
north.00799.h5
0.9974457215836526
north.01029.h5
1.0
north.01362.h5
1.0
north.00951.h5
1.0
north.00306.h5
1.0
north.01013.h5
1.0
north.01391.h5
1.0
north.00503.h5
0.9866666666666667
north.01200.h5
1.0
north.00821.h5
0.997564935064935
north.00276.h5
1.0
north.00944.h5
1.0
nor

0.9953271028037384
north.00809.h5
0.998589562764457
north.00559.h5
0.9983579638752053
north.01052.h5
0.9984928409947249
north.00773.h5
1.0
north.00339.h5
1.0
north.01183.h5
1.0
north.00332.h5
0.9982363315696648
north.01377.h5
0.9986225895316805
north.00507.h5
0.9957805907172996
north.00439.h5
1.0
north.01384.h5
0.9991326973113617
north.00553.h5
0.9952941176470588
north.00681.h5
1.0
north.01198.h5
1.0
north.01364.h5
1.0
north.01330.h5
0.9948979591836735
north.00323.h5
1.0
north.01050.h5
0.9937694704049844
north.00331.h5
1.0
north.00309.h5
0.996078431372549
north.00328.h5
1.0
north.01383.h5
0.9969230769230769
north.00476.h5
1.0
north.00946.h5
1.0
north.01077.h5
0.9979381443298969
north.00786.h5
0.9963065558633426
north.01065.h5
0.9985754985754985
north.01036.h5
0.997164461247637
north.01011.h5
1.0
north.01054.h5
0.9990671641791045
north.00529.h5
0.9988505747126437
north.00965.h5
0.9959183673469387
north.00518.h5
0.9949494949494949
north.00700.h5
0.9949367088607595
north.01295.h5
1.0
nort

In [17]:
df_goodspec['chisq_rerun'] = chisqrerun

<ipython-input-17-843f4aea3a03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goodspec['chisq_rerun'] = chisqrerun


In [18]:
df_goodspec.filter(items=['chisq', 'chisq_rerun'])

,chisq,chisq_rerun
2,18.058500,18.153484
3,27.060320,27.708298
7,0.165382,0.043352
9,0.026190,0.990230
10,0.376068,0.131778
...,...,...
596290,0.319424,0.032164
596291,2.328202,2.015753
596296,0.027911,0.028205
596298,inf,inf


In [19]:
df_goodspec.to_pickle('specmatched_northrerun.pkl')

## Section 1: 
### How many objects are not stars before cuts?

In [22]:
df_goodspec['chisq'] = df_goodspec['chisq_rerun']

<ipython-input-22-13dcd7c4c576>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goodspec['chisq'] = df_goodspec['chisq_rerun']


In [23]:
df_goodspec['chisq']

2         18.153484
3         27.708298
7          0.043352
9          0.990230
10         0.131778
            ...    
596290     0.032164
596291     2.015753
596296     0.028205
596298          inf
596305     0.073052
Name: chisq, Length: 210713, dtype: float64

In [24]:
spec_utils.print_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'])

Number of Stars = 175597, Fraction(%) = 83.3347, Ratio(obj:star)*100 = 100
Number of Galaxies = 299, Fraction(%) = 0.1419, Ratio(obj:star)*100 = 0.1703
Number of QSOs = 34817, Fraction(%) = 16.5234, Ratio(obj:star)*100 = 19.8278


### After cuts: 16a

In [25]:
nbtest_dir = '../../notebooks_test/'

In [26]:
def combine_cuts(df_input, cuts_list):
    cutfuncs = [getattr(astro_cuts, ctup[0]) for ctup in cuts_list]
    masklist = []
    for c, ctup in enumerate(cuts_list):
        if len(ctup)==1:
            masklist.append(cutfuncs[c](df_input))
        else:
            masklist.append(cutfuncs[c](df_input, **ctup[1]))
        print('Cut{} {} {}'.format(c, np.sum(masklist[c]), np.sum(masklist[c])/len(masklist[c])))
    final_cut = reduce(np.logical_and, masklist) #intersection of all cuts
    print('Effective Cut = {:.3f}'.format(np.sum(final_cut)/len(final_cut)))
    return final_cut

In [27]:
os.listdir('models')

['.ipynb_checkpoints',
 'svm_sdss_quasarsep_unbalanced.joblib',
 'svm_panstarrs_quasarsep_balanced.joblib',
 'svm_sdss_quasarsep_balanced_new.joblib',
 'svm_sdss_quasarsep_balanced.joblib']

In [29]:
wisemodel = joblib.load('models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('models/svm_sdss_quasarsep_balanced_new.joblib')

cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]
final_cut = combine_cuts(df_goodspec, cuts_list) #intersection of all cuts
dfpost = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfpost['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 210106 0.997119304456773
Cut4 173743 0.8245480819882969
Cut5 156011 0.740395704109381
Effective Cut = 0.659
Number of Stars = 138797, Fraction(%) = 99.9302, Ratio(obj:star)*100 = 100
Number of Galaxies = 51, Fraction(%) = 0.0367, Ratio(obj:star)*100 = 0.0367
Number of QSOs = 46, Fraction(%) = 0.0331, Ratio(obj:star)*100 = 0.0331


In [30]:
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfpost['sdss_dr17_specobj.CLASS'])

StarFracChange = 16.595 (%)
GalFracChange = -0.105 (%)
QSOFracChange = -16.490 (%)
Stars thrown out = -36800


In [31]:
nbtest_dir

'../../notebooks_test/'

### Ablation Tests: which cut is doing what?

In [32]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}) ]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Effective Cut = 0.926
Number of Stars = 162960, Fraction(%) = 83.5354, Ratio(obj:star)*100 = 100
Number of Galaxies = 280, Fraction(%) = 0.1435, Ratio(obj:star)*100 = 0.1718
Number of QSOs = 31839, Fraction(%) = 16.3211, Ratio(obj:star)*100 = 19.5379
StarFracChange = 0.201 (%)
GalFracChange = 0.002 (%)
QSOFracChange = -0.202 (%)
Stars thrown out = -12637


In [34]:
cuts_list =  [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Effective Cut = 0.761
Number of Stars = 158629, Fraction(%) = 98.9082, Ratio(obj:star)*100 = 100
Number of Galaxies = 190, Fraction(%) = 0.1185, Ratio(obj:star)*100 = 0.1198
Number of QSOs = 1561, Fraction(%) = 0.9733, Ratio(obj:star)*100 = 0.9841
StarFracChange = 15.574 (%)
GalFracChange = -0.023 (%)
QSOFracChange = -15.550 (%)
Stars thrown out = -16968


In [36]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',)]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 210106 0.997119304456773
Effective Cut = 0.759
Number of Stars = 158311, Fraction(%) = 98.9388, Ratio(obj:star)*100 = 100
Number of Galaxies = 137, Fraction(%) = 0.0856, Ratio(obj:star)*100 = 0.0865
Number of QSOs = 1561, Fraction(%) = 0.9756, Ratio(obj:star)*100 = 0.9860
StarFracChange = 15.604 (%)
GalFracChange = -0.056 (%)
QSOFracChange = -15.548 (%)
Stars thrown out = -17286


In [39]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 210106 0.997119304456773
Cut4 173743 0.8245480819882969
Effective Cut = 0.741
Number of Stars = 155897, Fraction(%) = 99.8003, Ratio(obj:star)*100 = 100
Number of Galaxies = 109, Fraction(%) = 0.0698, Ratio(obj:star)*100 = 0.0699
Number of QSOs = 203, Fraction(%) = 0.1300, Ratio(obj:star)*100 = 0.1302
StarFracChange = 16.466 (%)
GalFracChange = -0.072 (%)
QSOFracChange = -16.393 (%)
Stars thrown out = -19700


In [40]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 210106 0.997119304456773
Cut4 173743 0.8245480819882969
Cut5 156011 0.740395704109381
Effective Cut = 0.659
Number of Stars = 138797, Fraction(%) = 99.9302, Ratio(obj:star)*100 = 100
Number of Galaxies = 51, Fraction(%) = 0.0367, Ratio(obj:star)*100 = 0.0367
Number of QSOs = 46, Fraction(%) = 0.0331, Ratio(obj:star)*100 = 0.0331
StarFracChange = 16.595 (%)
GalFracChange = -0.105 (%)
QSOFracChange = -16.490 (%)
Stars thrown out = -36800


### All cuts except

#### the WISE cut

In [41]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('parallax_nan_edr3',), ('sdss_uvcut', {'model': smodel}), 
             ('bayestar_chisq', {'maxchisq': 3.0})]

final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 210106 0.997119304456773
Cut3 173743 0.8245480819882969
Cut4 156011 0.740395704109381
Effective Cut = 0.665
Number of Stars = 139957, Fraction(%) = 99.8858, Ratio(obj:star)*100 = 100
Number of Galaxies = 57, Fraction(%) = 0.0407, Ratio(obj:star)*100 = 0.0407
Number of QSOs = 103, Fraction(%) = 0.0735, Ratio(obj:star)*100 = 0.0736
StarFracChange = 16.551 (%)
GalFracChange = -0.101 (%)
QSOFracChange = -16.450 (%)
Stars thrown out = -35640


#### Redchisq cut

In [42]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel})]


final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 210106 0.997119304456773
Cut4 173743 0.8245480819882969
Effective Cut = 0.741
Number of Stars = 155897, Fraction(%) = 99.8003, Ratio(obj:star)*100 = 100
Number of Galaxies = 109, Fraction(%) = 0.0698, Ratio(obj:star)*100 = 0.0699
Number of QSOs = 203, Fraction(%) = 0.1300, Ratio(obj:star)*100 = 0.1302
StarFracChange = 16.466 (%)
GalFracChange = -0.072 (%)
QSOFracChange = -16.393 (%)
Stars thrown out = -19700


#### SDSS cut

In [43]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('bayestar_chisq', {'maxchisq': 3.0})]


final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 210106 0.997119304456773
Cut4 156011 0.740395704109381
Effective Cut = 0.664
Number of Stars = 139725, Fraction(%) = 99.7907, Ratio(obj:star)*100 = 100
Number of Galaxies = 61, Fraction(%) = 0.0436, Ratio(obj:star)*100 = 0.0437
Number of QSOs = 232, Fraction(%) = 0.1657, Ratio(obj:star)*100 = 0.1660
StarFracChange = 16.456 (%)
GalFracChange = -0.098 (%)
QSOFracChange = -16.358 (%)
Stars thrown out = -35872


#### Parallax_nan_edr3

In [44]:
cuts_list = [('distmod_median_cut_corr', {'minpc': 400}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut_limiting', {'model': wisemodel}),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]


final_cut = combine_cuts(df_goodspec, cuts_list)
dfalt = df_goodspec.iloc[final_cut, :]
spec_utils.print_obj_distribution(dfalt['sdss_dr17_specobj.CLASS'])
spec_utils.print_delta_obj_distribution(df_goodspec['sdss_dr17_specobj.CLASS'], dfalt['sdss_dr17_specobj.CLASS'])

Cut0 205550 0.9754974776117278
Cut1 200235 0.9502735948897315
Cut2 172977 0.8209128055696611
Cut3 173743 0.8245480819882969
Cut4 156011 0.740395704109381
Effective Cut = 0.660
Number of Stars = 139059, Fraction(%) = 99.9224, Ratio(obj:star)*100 = 100
Number of Galaxies = 62, Fraction(%) = 0.0446, Ratio(obj:star)*100 = 0.0446
Number of QSOs = 46, Fraction(%) = 0.0331, Ratio(obj:star)*100 = 0.0331
StarFracChange = 16.588 (%)
GalFracChange = -0.097 (%)
QSOFracChange = -16.490 (%)
Stars thrown out = -36538


In [29]:
def pan_wise_cuts(df_input):
    #riz < 0.2
    psmerrs = df_input[['mag_err_r', 'mag_err_i', 'mag_err_z']].to_numpy()     
    cut1 = np.all(psmerrs<0.2, axis=1)
    #W1<0.2
    w1merr = df_input['allwise.w1sigmpro'].to_numpy()
    cut2 = w1merr<0.2
    #w1rchi2
    cut3 = df_input['allwise.w1rchi2'].to_numpy()<2
    #detected in w1
    cut4 = df_input['allwise.w1mpro'].to_numpy()!=0
    print('C1: {:.2f} C2: {:.2f} C3: {:.2f} C4: {:.2f}'.format(np.sum(cut1)/len(cut1), np.sum(cut2)/len(cut2), np.sum(cut3)/len(cut3), np.sum(cut4)/len(cut4)))
    
    effcut = cut1 * cut2 * cut3 * cut4
    
    return effcut

In [30]:
wmask = pan_wise_cuts(df_goodspec)

C1: 1.00 C2: 0.99 C3: 0.99 C4: 0.77


In [31]:
w1vals = df_goodspec.iloc[wmask, :]['allwise.w1mpro'].to_numpy()

In [35]:
np.percentile(w1vals, 97)

17.475000381469727